# Ham ya da Spam?

🎯 Bu görevin amacı, e-postaları **spam (1)** veya **normal e-posta (0)** olarak sınıflandırmaktır.

🧹 İlk olarak, bu metin verilerine **temizleme (cleaning)** teknikleri uygulanacaktır.

👩🏻‍🔬 Ardından, temizlenmiş metinler **sayısal bir gösterime** dönüştürülecektir.

✉️ Son olarak, her bir e-postayı spam mı yoksa normal mi olduğunu sınıflandırmak için  
***Multinomial Naive Bayes*** modeli uygulanacaktır.


## (0) NTLK kütüphanesi (Doğal Dil Araç Seti)

In [1]:
!pip install nltk

In [2]:
# nltk'yi ilk kez içe aktarırken, birkaç yerleşik kütüphaneyi de indirmemiz gerekir.

import nltk

nltk.download('stopwords')
nltk.download('punkt')      # nltk<3.9.0 için
nltk.download('punkt_tab')  # nltk>=3.9.0 için
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/egkpnr/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/egkpnr/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /home/egkpnr/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /home/egkpnr/nltk_data...
[nltk_data] Downloading package omw-1.4 to /home/egkpnr/nltk_data...


True

In [3]:
import pandas as pd

df = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/ham_spam_emails.csv")
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


## (1) (Metin) veri setinin temizlenmesi

Veri kümesi, ham [0] veya spam [1] olarak sınıflandırılan e-postalardan oluşur. Tahmin modelini eğitmeden önce veri kümesini temizlemeniz gerekir.

### (1.1) Noktalama İşaretlerini Kaldır

❓ Noktalama işaretlerini kaldırmak için bir işlev oluşturun. Bunu `text` sütununa uygulayın ve çıktıyı `clean_text` adlı veri çerçevesinin yeni bir sütununa ekleyin. ❓

In [5]:
import string

# Noktalama işaretlerini kaldırma fonksiyonu
def remove_punctuation(text):
    # string.punctuation, tüm noktalama işaretlerini içeren bir dizidir
    punctuation_free = "".join([i for i in text if i not in string.punctuation])
    return punctuation_free

# Fonksiyonu 'text' sütununa uygulama ve yeni sütuna kaydetme
df['clean_text'] = df['text'].apply(lambda x: remove_punctuation(x))

# Sonucu kontrol etme
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,Subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,1,Subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,1,Subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,1,Subject 4 color printing special request addi...
4,"Subject: do not have money , get software cds ...",1,Subject do not have money get software cds fr...


### (1.2) Küçük Harf

❓ Metni küçük harfe çeviren bir işlev oluşturun. Bunu `clean_text`'e uygulayın ❓

In [6]:
# Küçük harfe çevirme fonksiyonu
def lower_text(text):
    return text.lower()

# Fonksiyonu 'clean_text' sütununa uygulama
df['clean_text'] = df['clean_text'].apply(lambda x: lower_text(x))

# Sonucu kontrol etme
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,1,subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,1,subject 4 color printing special request addi...
4,"Subject: do not have money , get software cds ...",1,subject do not have money get software cds fr...


### (1.3) Sayıları Kaldır

❓ Metinden sayıları kaldırmak için bir işlev oluşturun. Bunu `clean_text`'e uygulayın ❓

In [7]:
# Sayıları kaldırma fonksiyonu
def remove_numbers(text):
    # Karakter rakam değilse tut, rakamsa at
    return "".join([i for i in text if not i.isdigit()])

# Fonksiyonu 'clean_text' sütununa uygulama
df['clean_text'] = df['clean_text'].apply(lambda x: remove_numbers(x))

# Sonucu kontrol etme
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,1,subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,1,subject color printing special request addit...
4,"Subject: do not have money , get software cds ...",1,subject do not have money get software cds fr...


### (1.4) StopWords'ü kaldırın

❓ Metinden durdurma kelimelerini kaldırmak için bir işlev oluşturun. Bunu `clean_text`'e uygulayın. ❓

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# İngilizce etkisiz kelimeleri (stopwords) yükleyelim
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    # Metni kelimelerine (token) ayıralım
    tokens = word_tokenize(text)
    # Stopword listesinde OLMAYAN kelimeleri seçelim
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Kelimeleri tekrar birleştirerek metin haline getirelim
    return " ".join(filtered_tokens)

# Fonksiyonu 'clean_text' sütununa uygulayalım
df['clean_text'] = df['clean_text'].apply(lambda x: remove_stopwords(x))

# Sonucu kontrol edelim
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible corporate ident...
1,Subject: the stock trading gunslinger fanny i...,1,subject stock trading gunslinger fanny merrill...
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new homes made easy im wa...
3,Subject: 4 color printing special request add...,1,subject color printing special request additio...
4,"Subject: do not have money , get software cds ...",1,subject money get software cds software compat...


### (1.5) Lemmatize

❓ Metni lemmatize etmek için bir fonksiyon oluşturun. Çıktının bir kelime listesi değil, tek bir dize olduğundan emin olun. Bunu `clean_text`'e uygulayın. ❓

In [9]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Lemmatizer nesnesini başlatma
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    # Metni kelimelerine ayır
    tokens = word_tokenize(text)
    
    # Her kelimeyi köküne indir (Lemmatization)
    # Not: Parametre verilmezse varsayılan olarak isim (noun) kabul eder.
    lemma_words = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Listeyi tekrar tek bir string (dize) haline getir
    return " ".join(lemma_words)

# Fonksiyonu 'clean_text' sütununa uygulama
df['clean_text'] = df['clean_text'].apply(lambda x: lemmatize_text(x))

# Sonucu kontrol etme
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible corporate ident...
1,Subject: the stock trading gunslinger fanny i...,1,subject stock trading gunslinger fanny merrill...
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new home made easy im wan...
3,Subject: 4 color printing special request add...,1,subject color printing special request additio...
4,"Subject: do not have money , get software cds ...",1,subject money get software cd software compati...


## (2) Bag-of-Words Modellemesi

### (2.1) Metin verilerini sayılara dönüştürme

❓ `clean_text`'i varsayılan CountVectorizer ile Bag-of-Words temsiline vektörleştirin. `X_bow` olarak kaydedin. ❓

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer nesnesini başlatma
vectorizer = CountVectorizer()

# Metni vektörleştirme (fit ve transform işlemi bir arada)
X_bow = vectorizer.fit_transform(df['clean_text'])

# Sonuç hakkında bilgi (isteğe bağlı)
print(f"Oluşturulan matrisin boyutu: {X_bow.shape}")

Oluşturulan matrisin boyutu: (5728, 30988)


### (2.2) Çok terimli Naive Bayes Modellemesi

❓ MultinomialNB modelini bag-of-words verileriyle çapraz doğrulayın. Modelin doğruluğunu puanlayın. ❓

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

# Multinomial Naive Bayes modelini başlatma
nb_model = MultinomialNB()

# Çapraz doğrulama ile modeli değerlendirme
# cv=5: Veriyi 5 parçaya böler, her seferinde 4'ü ile eğitip 1'i ile test eder.
# scoring='accuracy': Başarı metriği olarak 'doğruluk' oranını kullanır.
scores = cross_val_score(nb_model, X_bow, df['spam'], cv=5, scoring='accuracy')

# Sonuçları yazdırma
print("Her bir katlamanın doğruluk puanları:", scores)
print("Ortalama Doğruluk Puanı:", scores.mean())

Her bir katlamanın doğruluk puanları: [0.98691099 0.9895288  0.991274   0.98777293 0.99213974]
Ortalama Doğruluk Puanı: 0.9895252901681946


🏁 Tebrikler!

💾 Not defterinizi git add/commit/push yapmayı unutmayın...

🚀 ... ve bir sonraki challenge'a geçin!